In [ ]:
%cd ../

In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import defaultdict

In [ ]:
def visualize_response(data_basename, filename, measure, ax=None):
    data = defaultdict(list)
    for noise_rate in [0.1, 0.2, 0.3, 0.4, 0.5]:
        data_dir_ = data_basename + str(noise_rate) + "/"
        results = json.load(open(data_dir_ + filename, "r"))
        for result in results:
            val = result[measure]
            data["measure"] += [measure]
            data["noise_rate"] += [noise_rate]
            data["value"] += [val]

    df = pd.DataFrame(data)
    sns.set(style="whitegrid")
    if ax is None:
        _, ax = plt.subplots(1, 1, figsize=(5, 5))
    sns.boxplot(x="noise_rate", y="value", hue="measure", data=df, ax=ax)
    ax.set_title(measure)
    sns.despine()
    # return fig


In [ ]:
f, axes = plt.subplots(1, 3, figsize=(12, 5))
visualize_response("data/results/response_analysis/small_spatial_and_label_noise/elem_noise_rate_", "results.json", "maximum_iou", ax=axes[0])
visualize_response("data/results/response_analysis/small_spatial_and_label_noise/elem_noise_rate_", "mmd.json", "mmd", ax=axes[1])
visualize_response("data/results/response_analysis/small_spatial_and_label_noise/elem_noise_rate_", "fid.json", "fid", ax=axes[2])
axes[0].set_ylabel("Value")
axes[1].set_ylabel("")
axes[2].set_ylabel("")
axes[0].set_xlabel("Noise Injection Rate")
axes[1].set_xlabel("Noise Injection Rate")
axes[2].set_xlabel("Noise Injection Rate")
axes[0].set_title("Max.IOU")
axes[1].set_title("LaTSim-MMD")
axes[2].set_title("FID")
plt.savefig("figs/response_analysis/small_spatial_and_label_noise_response_analysis.pdf", bbox_inches="tight")

In [ ]:
from experiments.utils import layout_to_svg_image

def display_noise_examples(num_examples=5):
    h_size, w_size = 10/2, 7/2
    f, axes = plt.subplots(num_examples, 6, figsize=(6*w_size, h_size*num_examples))
    ref_file = "data/cvpr2023_json/datasets/rico25/val.json"
    org_data = json.load(open(ref_file, "r"))
    keys = np.random.choice(len(org_data["annotations"].keys()), num_examples)
    for i, key in enumerate(keys):
        layout = list(org_data["annotations"].values())[key]
        layout_to_svg_image(layout, org_data["meta"], size=(h_size, w_size), border_cfg={"color": (0.7, 0.7, 0.7), "width": 1}, show_labels=False, ax=axes[i,0])
    axes[0,0].set_title("Original")
    base_dir = "data/results/response_analysis/small_spatial_and_label_noise"
    for i, noise in enumerate([0.1, 0.2, 0.3, 0.4, 0.5]):
        data_dir = base_dir + "/elem_noise_rate_" + str(noise)
        file = data_dir + "/data/0.json"
        data = json.load(open(file, "r"))
        for j, key in enumerate(keys):
            layout = data[key]
            layout_to_svg_image(layout, org_data["meta"], size=(h_size, w_size), border_cfg={"color": (0.7, 0.7, 0.7), "width": 1}, show_labels=False, ax=axes[j,i+1])
        axes[0,i+1].set_title("Noise Rate: " + str(noise))
    return f

f = display_noise_examples(3)
plt.savefig("figs/response_analysis/small_spatial_and_label_noise_examples.pdf", bbox_inches="tight")